### **Objective: To predict the behaviour of the customers to retain them**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_rows',3000)
pd.set_option('display.max_columns',3000)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# 1. READING AND UNDERSTANDING THE DATA

In [ ]:
# Data Reading
churn_data = pd.read_csv("/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")

In [ ]:
churn_data.head()

In [ ]:
# Data Inspection
churn_data.info()

#### The above data shows no missing values in the dataset.

In [ ]:
# Shape of the dataset
churn_data.shape

#### Therefore there are 7043 rows and 21 columns in the data set.

In [ ]:
# Statistical aspects of the numerical columns of the dataset
churn_data.describe()

# 2. DATA PREPARATION

In [ ]:
# Observing the data in the dataset to search for binary variables
churn_data.head()

#### The binary variables are as follows:
- Partner,Dependents,PhoneService,OnlineSecurity, OnlineBackup, DeviceProtection,TechSupport, StreamingTV,StreamingMovies,PaperlessBilling,Churn	 

In [ ]:
# Converting binary variables (Yes/No) to 0 or 1

In [ ]:
varlist = ['Partner','Dependents','PhoneService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','PaperlessBilling','Churn']

In [ ]:
# function to map binary columns

def binary_mapping(x):
    return x.map({'Yes':1,'No':0})

In [ ]:
churn_data[varlist]=churn_data[varlist].apply(binary_mapping)

In [ ]:
churn_data.head()

In [ ]:
# Creating dummy variables with some of the categorical variables with multiple levels
dummy1=pd.get_dummies(churn_data[['Contract','PaymentMethod','gender','InternetService']], drop_first=True)

In [ ]:
churn_data = pd.concat([churn_data,dummy1],axis=1)

In [ ]:
churn_data.head()

In [ ]:
# Creating dummy variables for the remaining categorical variables and dropping the level with big names.


ml = pd.get_dummies(churn_data['MultipleLines'], prefix='MultipleLines')
# Dropping MultipleLines_No phone service column
ml1 = ml.drop(['MultipleLines_No phone service'], 1)
#Adding the results to the master dataframe
churn_data = pd.concat([churn_data,ml1], axis=1)


In [ ]:
churn_data.head()

In [ ]:
# Dropping the redundant variables
churn_data=churn_data.drop(['Contract','PaymentMethod','gender','InternetService','MultipleLines'],1)

In [ ]:
churn_data.head()

In [ ]:
# Observing the data types of the variables
churn_data.info()

In [ ]:
# Converting TotalCharges into float as it is a numerical column
churn_data['TotalCharges'] = pd.to_numeric(churn_data.TotalCharges, errors='coerce')


In [ ]:
# Observing the change
churn_data.info()

In [ ]:
# Checking for outliers in numerical columns
num_churn_data=churn_data[['tenure','MonthlyCharges','TotalCharges']]

In [ ]:
# Checking whether the numbers are gradually increasing
num_churn_data.describe(percentiles=[0.25,0.50,0.75,0.90,0.95,0.99])

#### The numbers seems to be gradually increasing. Therefore there are no outliers.

# MISSING VALUE TREATMENT

In [ ]:
# Checking for missing values
churn_data.isnull().sum()

In [ ]:
# Checking the percentage of missing values
round(100*(churn_data.isnull().sum()/len(churn_data.index)),2)

### Therefore the columns haveing missing values are as follows:
- OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
### So the above columns missing value needs to be treated
- TotalCharges column contains very less missing values so they can be dropped

In [ ]:
# Checking the rows against the missing values of Online Security
churn_data[np.isnan(churn_data['OnlineSecurity'])]

In [ ]:
# Checking the rows against the missing values of OnlineBackup
churn_data[np.isnan(churn_data['OnlineBackup'])]

In [ ]:
# Checking the rows against the missing values of DeviceProtection
churn_data[np.isnan(churn_data['DeviceProtection'])]

In [ ]:
# Checking the rows against the missing values of TechSupport
churn_data[np.isnan(churn_data['TechSupport'])]

In [ ]:
# Checking the rows against the missing values of StreamingTV
churn_data[np.isnan(churn_data['StreamingTV'])]

In [ ]:
# Checking the rows against the missing values of StreamingTV
churn_data[np.isnan(churn_data['StreamingMovies'])]

#### From the above data we can assume that all missing values are 0 as all these variables are related to internet and these customers have not availed internet as all InternetService_No variables against them appears to be 1.

In [ ]:
# To check which one has maximum count 0 or 1 
churn_data['OnlineSecurity'].value_counts()

In [ ]:
# ,'OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies'
# To check which one has maximum count 0 or 1 
churn_data['OnlineBackup'].value_counts()

In [ ]:
churn_data['DeviceProtection'].value_counts()

In [ ]:
churn_data['TechSupport'].value_counts()

In [ ]:
churn_data['StreamingTV'].value_counts()

In [ ]:
churn_data['StreamingTV'].mode()

In [ ]:
churn_data['StreamingMovies'].value_counts()

In [ ]:
churn_data['StreamingMovies'].mode()

#### Since in all the above variables 0 appears most of the time we will replace the missing values of all the above columns with 0.

In [ ]:
churn_data[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']]=churn_data[['OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies']].fillna(0)

In [ ]:
# Checking the sum of missing values in the columns now
churn_data.isnull().sum()

In [ ]:
# Checking the percentage of missing values in each of the columns now
round(100*(churn_data.isnull().sum()/churn_data.shape[0]),2)

#### From the above data we can observe that the column TotalCharges have negligible number and percentage of missing values.

In [ ]:
# Dropping the rows where TotalCharges have missing values
clean_data=churn_data[~np.isnan(churn_data['TotalCharges'])]

In [ ]:
# Checking the number of missing values in all the columns
clean_data.isnull().sum()

#### The data is clean now. Thus we can proceed with the next process.

# 3. TRAIN-TEST SPLIT

In [ ]:
# importing train test split library
from sklearn.model_selection import train_test_split

In [ ]:
# Putting feature variable to X
X = clean_data.drop(['Churn','customerID'], axis=1)

X.head()

In [ ]:
# Putting response variable in y
y = clean_data['Churn']
y.head()

In [ ]:
# Splitting the data into train and test
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

# 4. FEATURE SCALING

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# scaling numerical variables
scaler = StandardScaler()

X_train[['tenure','MonthlyCharges','TotalCharges']] = scaler.fit_transform(X_train[['tenure','MonthlyCharges','TotalCharges']])

X_train.head()

In [ ]:
### Checking the Churn Rate
churn = (sum(clean_data['Churn'])/len(clean_data['Churn'].index))*100
churn

## This shows there is 27% of churn rate as per existing data.

# 5. CHECKING CORRELATIONS

In [ ]:
# Importing matplotlib and seaborn
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# Let's see the correlation matrix 
plt.figure(figsize = (20,10))        # Size of the figure
sns.heatmap(clean_data.corr(),annot = True)
plt.show()

In [ ]:
# Dropping highly correlated dummy variable
X_test = X_test.drop(['MultipleLines_No'], 1)
X_train = X_train.drop(['MultipleLines_No'], 1)

In [ ]:
# Checking the correlation matrix
plt.figure(figsize = (20,10))
sns.heatmap(X_train.corr(),annot = True)
plt.show()

# 6. MODEL BUILDING

In [ ]:
# library required for building the model
import statsmodels.api as sm

In [ ]:
# LOGISTIC REGRESSION MODEL
logm1 = sm.GLM(y_train,(sm.add_constant(X_train)), family = sm.families.Binomial())
logm1.fit().summary()

# 7. FEATURE SELECTION USING RFE

In [ ]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()

In [ ]:
from sklearn.feature_selection import RFE
rfe = RFE(logreg, 15)             # running RFE with 15 variables as output
rfe = rfe.fit(X_train, y_train)

In [ ]:
rfe.support_

In [ ]:
list(zip(X_train.columns, rfe.support_, rfe.ranking_))

In [ ]:
col = X_train.columns[rfe.support_]

In [ ]:
col

In [ ]:
X_train.columns[~rfe.support_]

#### Assessing the model using StatsModel

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm2 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm2.fit()
res.summary()

In [ ]:
# Getting the predicted values on the train set
y_train_pred = res.predict(X_train_sm)
y_train_pred[:10]

In [ ]:
y_train_pred_final = pd.DataFrame({'Churn':y_train.values, 'Churn_Prob':y_train_pred})
y_train_pred_final['CustID'] = y_train.index
y_train_pred_final.head()

In [ ]:
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)

# Let's see the head
y_train_pred_final.head()

In [ ]:
from sklearn import metrics

In [ ]:
# Confusion matrix 
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
print(confusion)

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

## Checking VIFs

In [ ]:
# Check for the VIF values of the feature variables. 
from statsmodels.stats.outliers_influence import variance_inflation_factor

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### Here Monthly Charges has the highest VIF. So we will drop that column.

In [ ]:
col = col.drop('MonthlyCharges', 1)


In [ ]:
col

In [ ]:
# Let's re-run the model using the selected variables
X_train_sm = sm.add_constant(X_train[col])
logm3 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm3.fit()
res.summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### Next we will remove Total Charges with high VIF.

In [ ]:
col = col.drop('TotalCharges', 1)

In [ ]:
X_train_sm = sm.add_constant(X_train[col])
logm4 = sm.GLM(y_train,X_train_sm, family = sm.families.Binomial())
res = logm4.fit()
res.summary()

In [ ]:
# Create a dataframe that will contain the names of all the feature variables and their respective VIFs
vif = pd.DataFrame()
vif['Features'] = X_train[col].columns
vif['VIF'] = [variance_inflation_factor(X_train[col].values, i) for i in range(X_train[col].shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = "VIF", ascending = False)
vif

#### Now all the variables are highly significant.

In [ ]:
y_train_pred = res.predict(X_train_sm).values.reshape(-1)

In [ ]:
y_train_pred[:10]

In [ ]:
y_train_pred_final['Churn_Prob'] = y_train_pred

In [ ]:
# Creating new column 'predicted' with 1 if Churn_Prob > 0.5 else 0
y_train_pred_final['predicted'] = y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.5 else 0)
y_train_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
print(metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted))

# 8. CONFUSION MATRIX

In [ ]:
# Let's take a look at the confusion matrix again 
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
confusion

In [ ]:
# Actual/Predicted     not_churn    churn
        # not_churn        3269      366
        # churn            595       692 

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.predicted)

In [ ]:
# Metrics beyond accuracy
TP = confusion[1,1] # true positive 
TN = confusion[0,0] # true negatives
FP = confusion[0,1] # false positives
FN = confusion[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FP)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate - predicting churn when customer does not have churned
print(FP/ float(TN+FP))

# PLOTTING ROC CURVE

In [ ]:
def draw_roc( actual, probs ):
    fpr, tpr, thresholds = metrics.roc_curve( actual, probs,
                                              drop_intermediate = False )
    auc_score = metrics.roc_auc_score( actual, probs )
    plt.figure(figsize=(5, 5))
    plt.plot( fpr, tpr, label='ROC curve (area = %0.2f)' % auc_score )
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate or [1 - True Negative Rate]')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver operating characteristic example')
    plt.legend(loc="lower right")
    plt.show()

    return None

In [ ]:
fpr, tpr, thresholds = metrics.roc_curve( y_train_pred_final.Churn, y_train_pred_final.Churn_Prob, drop_intermediate = False )

In [ ]:
draw_roc(y_train_pred_final.Churn, y_train_pred_final.Churn_Prob)

# 9. FINDING THE OPTIMAL CUT OFF POINT

In [ ]:
# Let's create columns with different probability cutoffs 
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.Churn_Prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
# Let's plot accuracy sensitivity and specificity for various probabilities.
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

#### From the curve above, 0.3 is the optimum point to take it as a cutoff probability.

In [ ]:
y_train_pred_final['final_predicted'] = y_train_pred_final.Churn_Prob.map( lambda x: 1 if x > 0.3 else 0)

y_train_pred_final.head()

# 10. EVALUATION OF THIS MODEL WHICH IS CREATED ACCORDING TO THE OPTIMAL CUT OFF

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_train_pred_final.Churn, y_train_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Calculate false postive rate - predicting churn when customer does not have churned
print(FP/ float(TN+FP))

In [ ]:
# Positive predictive value 
print (TP / float(TP+FP))

In [ ]:
# Negative predictive value
print (TN / float(TN+ FN))

# 11. PRECISION AND RECALL

In [ ]:
confusion = metrics.confusion_matrix(y_train_pred_final.Churn, y_train_pred_final.predicted )
confusion

##### Precision
TP / TP + FP

In [ ]:
confusion[1,1]/(confusion[0,1]+confusion[1,1])

##### Recall
TP / TP + FN

In [ ]:
confusion[1,1]/(confusion[1,0]+confusion[1,1])

In [ ]:
from sklearn.metrics import precision_score, recall_score

In [ ]:
precision_score(y_train_pred_final.Churn, y_train_pred_final.predicted)

# PRECISION AND RECALL TRADEOFF

In [ ]:
from sklearn.metrics import precision_recall_curve

In [ ]:
y_train_pred_final.Churn, y_train_pred_final.predicted

In [ ]:
p, r, thresholds = precision_recall_curve(y_train_pred_final.Churn, y_train_pred_final.Churn_Prob)

In [ ]:
# Precision and Recall method of finding the optimal value
plt.plot(thresholds, p[:-1], "g-")
plt.plot(thresholds, r[:-1], "r-")
plt.show()

#### Therefore the optimal value is 0.42

# 12. MAKING PREDICTIONS ON THE TEST SET

In [ ]:
X_test[['tenure','MonthlyCharges','TotalCharges']] = scaler.transform(X_test[['tenure','MonthlyCharges','TotalCharges']])

In [ ]:
X_test = X_test[col]
X_test.head()

In [ ]:
X_test_sm = sm.add_constant(X_test)

In [ ]:
y_test_pred = res.predict(X_test_sm)

In [ ]:
y_test_pred[:10]

In [ ]:
# Converting y_pred to a dataframe which is an array
y_pred_1 = pd.DataFrame(y_test_pred)

In [ ]:
# Let's see the head
y_pred_1.head()

In [ ]:
# Converting y_test to dataframe
y_test_df = pd.DataFrame(y_test)

In [ ]:
# Putting CustID to index
y_test_df['CustID'] = y_test_df.index

In [ ]:
# Removing index for both dataframes to append them side by side 
y_pred_1.reset_index(drop=True, inplace=True)
y_test_df.reset_index(drop=True, inplace=True)

In [ ]:
# Appending y_test_df and y_pred_1
y_pred_final = pd.concat([y_test_df, y_pred_1],axis=1)

In [ ]:
y_pred_final.head()

In [ ]:
# Renaming the column 
y_pred_final= y_pred_final.rename(columns={ 0 : 'Churn_Prob'})

In [ ]:
# Let's see the head of y_pred_final
y_pred_final.head()

In [ ]:
y_pred_final['final_predicted'] = y_pred_final.Churn_Prob.map(lambda x: 1 if x > 0.42 else 0)

In [ ]:
y_pred_final.head()

In [ ]:
# Let's check the overall accuracy.
metrics.accuracy_score(y_pred_final.Churn, y_pred_final.final_predicted)

In [ ]:
confusion2 = metrics.confusion_matrix(y_pred_final.Churn, y_pred_final.final_predicted )
confusion2

In [ ]:
TP = confusion2[1,1] # true positive 
TN = confusion2[0,0] # true negatives
FP = confusion2[0,1] # false positives
FN = confusion2[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)

In [ ]:
# Precision
TP / float(TP+FP)

### The sensitivity that is the probability of yeses correctly converted to yeses is more in test set than training set.

# This shows that this is a good model.

# BUSINESS ANALYSIS

## Following is the future churn behaviour of the customers of the respective customer IDs:

In [ ]:
# predicted churn result set
y_pred_final